In [14]:
import sys
sys.path.append('/kaggle/input/required4')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, TensorDataset, random_split
from dataloader import MNIST  
from model import CNNClassifier  

def train(data, file_name, params, num_epochs=50, batch_size=128, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   
    model = CNNClassifier(params).to(device)


    train_loader = data.train_loader
    val_loader = data.validation_loader

    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-6, nesterov=True)

   
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in data.train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
              f"Train Acc: {correct/total:.4f}, Val Acc: {val_acc:.4f}")

   
    if file_name is not None:
        torch.save(model.state_dict(), file_name)

    return model

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total



 


In [15]:
print("Current working directory:", os.getcwd())

# List files in current directory
print("Files in current directory:", os.listdir('.'))


Current working directory: /kaggle/working
Files in current directory: ['models', '.virtual_documents', 'MNIST']


In [16]:
print(open('/kaggle/input/required4/dataloader.py').read())

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

class MNIST:
    def __init__(self, batch_size=128):
        # Define transformations
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        # Automatically downloads if not available
        full_train = datasets.MNIST(root='/kaggle/working', train=True, download=True, transform=transform)
        self.test_data = datasets.MNIST(root='/kaggle/working', train=False, download=True, transform=transform)

        # Split training and validation sets
        val_size = 5000
        train_size = len(full_train) - val_size
        self.train_data, self.validation_data = random_split(full_train, [train_size, val_size])

        # DataLoaders
        self.train_loader = DataLoader(self.train_data, batch_size=batch_size, shuffle=True)
        self.validation_loader = DataLoader(self.validation_d

In [17]:
if not os.path.isdir('models'):
    os.makedirs('models')

# Call training function
train(MNIST(), "models/example_classifier.pth", [32, 32, 64, 64, 200, 200], num_epochs=50)

Epoch 1/50, Loss: 1.1315, Train Acc: 0.6003, Val Acc: 0.9614
Epoch 2/50, Loss: 0.1358, Train Acc: 0.9594, Val Acc: 0.9774
Epoch 3/50, Loss: 0.0912, Train Acc: 0.9725, Val Acc: 0.9848
Epoch 4/50, Loss: 0.0700, Train Acc: 0.9789, Val Acc: 0.9844
Epoch 5/50, Loss: 0.0580, Train Acc: 0.9827, Val Acc: 0.9870
Epoch 6/50, Loss: 0.0492, Train Acc: 0.9848, Val Acc: 0.9894
Epoch 7/50, Loss: 0.0445, Train Acc: 0.9866, Val Acc: 0.9870
Epoch 8/50, Loss: 0.0383, Train Acc: 0.9885, Val Acc: 0.9866
Epoch 9/50, Loss: 0.0332, Train Acc: 0.9894, Val Acc: 0.9890
Epoch 10/50, Loss: 0.0305, Train Acc: 0.9902, Val Acc: 0.9890
Epoch 11/50, Loss: 0.0265, Train Acc: 0.9915, Val Acc: 0.9900
Epoch 12/50, Loss: 0.0252, Train Acc: 0.9919, Val Acc: 0.9888
Epoch 13/50, Loss: 0.0235, Train Acc: 0.9925, Val Acc: 0.9904
Epoch 14/50, Loss: 0.0206, Train Acc: 0.9936, Val Acc: 0.9898
Epoch 15/50, Loss: 0.0200, Train Acc: 0.9939, Val Acc: 0.9890
Epoch 16/50, Loss: 0.0184, Train Acc: 0.9942, Val Acc: 0.9910
Epoch 17/50, Loss

CNNClassifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)